In [1]:
import torch
import torch.onnx
import onnx
# import onnxruntime
import numpy as np
import time
from model import transfer_resNet, ResNet54, ResNet22, ResNet38, resnet50, MMNet
from audiodataset import CoswaraDataset, ConcatDataset, CoswaraDataset2
import random
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from train import train, validation, mm_train

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"
torch.cuda.is_available()

/opt/conda/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


True

In [2]:
import torch
torch.__version__

'1.7.0'

In [9]:
import sys
onnx_model = onnx.load('a_model.onnx')
print("Producer Name:", onnx_model.producer_name)
print("Producer Version:", onnx_model.producer_version)
print("Opset", onnx_model.opset_import[0])

Producer Name: pytorch
Producer Version: 1.7
Opset version: 9



In [2]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

In [3]:
# audio
a_model = transfer_resNet(2)
# image
i_model = resnet50(num_classes=2)

In [4]:
state_dict = torch.load('/model/save1/f_a_model_0.923_0.923')
a_model.load_state_dict(state_dict)

<All keys matched successfully>

In [5]:
# audio dataset
data = np.load('audios_full.npz', allow_pickle=True)
aX, ay = data['x'], data['y']
data = np.load('images_tr_full.npz', allow_pickle=True)
iX_tr, y_tr = data['x'], data['y']
data = np.load('images_test.npz', allow_pickle=True)
iX_te, y_te = data['x'], data['y']

batch_size = 96
test_params = {'batch_size': batch_size,
               'shuffle': False,
               'num_workers': 15}

seed = 20
seed_everything(seed)
aX_tr, aX_te, ay_tr, ay_te = train_test_split(aX, ay, test_size=0.2, shuffle=True, stratify=ay, random_state=seed)
cos_test_dataset = CoswaraDataset2(aX_te, ay_te, mode='test')
o_test_loader = DataLoader(cos_test_dataset, **test_params)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

test examples =  472 [439  33]


In [8]:
a_model.to(device)
val_metrics, confusion_matrix = validation(device, batch_size, 2, a_model, o_test_loader, 0, None)
val_metrics.avg('accuracy')

/opt/conda/lib/python3.6/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/opt/conda/lib/python3.6/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/opt/conda/lib/python3.6/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use retu

Validation
 SUMMARY EPOCH: 0	Sample:  472/  472	Loss:0.2743	Accuracy:0.92

Confusion Matrix
[[423.  16.]
 [ 20.  13.]]


0.923728813559322

In [15]:
# audio
a_model = transfer_resNet(2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
a_model.to(device)
optimal_batch_size = 64
a_input = torch.randn(optimal_batch_size, 1, 80, 401).cuda()

In [7]:
mmnet = MMNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mmnet.to(device)
# state_dict = torch.load('model/mmnet1')
# mmnet.load_state_dict(state_dict)
optimal_batch_size = 16
a_input = torch.randn(optimal_batch_size, 1, 80, 401).cuda() # input size - (batch_size, 1, time_steps, mel_bins)
i_input = torch.randn(optimal_batch_size, 3, 224, 224).cuda()

In [16]:
# pytorch Throughput - audio
a_model.eval()
repetitions=100
total_time = 0
pytorch_time = []
with torch.no_grad():
    for rep in range(repetitions):
        tic = time.time()
        _ = a_model(a_input)
        toc = time.time()
        curr_time = toc - tic
        total_time += curr_time
        pytorch_time.append(curr_time)
Throughput = (repetitions*optimal_batch_size)/total_time
print('Final Throughput : ',Throughput)
print(np.mean(pytorch_time))

Final Throughput :  493.3935372844218
0.12971390008926392


In [8]:
# pytorch Throughput - mmnet
mmnet.eval()
repetitions=100
total_time = 0
pytorch_time = []
with torch.no_grad():
    for rep in range(repetitions):
        tic = time.time()
        _ = mmnet([a_input, i_input])
        toc = time.time()
        curr_time = toc - tic
        total_time += curr_time
        pytorch_time.append(curr_time)
Throughput = (repetitions*optimal_batch_size)/total_time
print('Final Throughput : ',Throughput)
print(np.mean(pytorch_time))

Final Throughput :  191.89291193458865
0.08337983846664429


In [20]:
# pytorch 시간 - audio
a_model.eval()
pytorch_time = []
for i in range(100):
    with torch.no_grad():
        tic = time.time()
        outputs = a_model(a_input)
        toc = time.time()
        dur = toc - tic
        pytorch_time.append(dur)
print(np.mean(pytorch_time))

0.12996405363082886


In [9]:
# pytorch 시간 - mmnet
mmnet.eval()
pytorch_time = []
for i in range(100):
    with torch.no_grad():
        tic = time.time()
        outputs = mmnet([a_input, i_input])
        toc = time.time()
        dur = toc - tic
        pytorch_time.append(dur)
print(np.mean(pytorch_time))

0.01368621587753296


In [5]:
a_model.to('cuda')
a_model.eval()
a_input = torch.randn(1, 1, 80, 401).cuda() # input size - (batch_size, 1, mel_bins, time_steps)
dummy_input = a_input

input_names = [ "input"]
output_names = [ "a_output", "a_fe"]

torch.onnx.export(a_model, dummy_input, "a_model.onnx",
                input_names=input_names, 
                output_names=output_names,
#                 opset_version=11,
#                 verbose=True,
                export_params=True
                 )

In [2]:
mmnet = MMNet()

In [ ]:
# onnx - mmnet
mmnet.to('cuda')
mmnet.eval()
optimal_batch_size = 64
a_input = torch.randn(optimal_batch_size, 1, 80, 401).cuda() # input size - (batch_size, 1, time_steps, mel_bins)
i_input = torch.randn(optimal_batch_size, 3, 224, 224).cuda()
dummy_input = [a_input, i_input]

input_names = [ "a_input", "i_input" ]
output_names = [ "output" ]

torch.onnx.export(mmnet, dummy_input, "mmnet_64.onnx",
                input_names=input_names, 
                output_names=output_names,
                export_params=True)

In [4]:
import tensorrt as trt

onnx_file_name = 'mmnet_32.onnx'
tensorrt_file_name = 'mmnet_32.trt'
fp16_mode = True
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(EXPLICIT_BATCH)
parser = trt.OnnxParser(network, TRT_LOGGER)

builder.max_batch_size = 64
builder.max_workspace_size = (1 << 30)
builder.fp16_mode = fp16_mode

# Parse model file
print('Loading ONNX file from path {}...'.format(onnx_file_name))
with open(onnx_file_name, 'rb') as model:
    print('Beginning ONNX file parsing')
    if not parser.parse(model.read()):
        print('ERROR: Failed to parse the ONNX file.')
        for error in range(parser.num_errors):
            print (parser.get_error(error))
print('Completed parsing of ONNX file')

print('Building an engine; this may take a while...')
engine = builder.build_cuda_engine(network)
print('Completed creating engine')
buf = engine.serialize()
with open(tensorrt_file_name, 'wb') as f:
    f.write(buf)

Loading ONNX file from path mmnet_32.onnx...
Beginning ONNX file parsing
Completed parsing of ONNX file
Building an engine; this may take a while...
Completed creating engine


In [17]:
# 4. Load TensorRT File & TensorRT Inference #################
import tensorrt as trt
import pycuda.driver as cuda
device = cuda.Device(0)
context = device.make_context()

class HostDeviceMem(object):
    def __init__(self, host_mem, device_mem):
        self.host = host_mem
        self.device = device_mem
 
    def __str__(self):
        return "Host:\n" + str(self.host) + "\nDevice:\n" + str(self.device)
 
    def __repr__(self):
        return self.__str__()
    
# Allocates all buffers required for an engine, i.e. host/device inputs/outputs.
def allocate_buffers(engine):
    inputs = []
    outputs = []
    bindings = []
    stream = cuda.Stream()
    for binding in engine:
        size = trt.volume(engine.get_binding_shape(binding)) * 64 #engine.max_batch_size
        dtype = trt.nptype(engine.get_binding_dtype(binding))
        # Allocate host and device buffers
        host_mem = cuda.pagelocked_empty(size, dtype)
        device_mem = cuda.mem_alloc(host_mem.nbytes)
        # Append the device buffer to device bindings.
        bindings.append(int(device_mem))
        # Append to the appropriate list.
        if engine.binding_is_input(binding):
            inputs.append(HostDeviceMem(host_mem, device_mem))
        else:
            outputs.append(HostDeviceMem(host_mem, device_mem))
#     ctx.pop()
#     del ctx
    return inputs, outputs, bindings, stream

def trt_init():
    tensorrt_file_name = 'a_model.trt'
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    trt_runtime = trt.Runtime(TRT_LOGGER)

    with open(tensorrt_file_name, 'rb') as f:
        engine_data = f.read()
    engine = trt_runtime.deserialize_cuda_engine(engine_data)
    return engine

# This function is generalized for multiple inputs/outputs.
# inputs and outputs are expected to be lists of HostDeviceMem objects.
def do_inference(context, bindings, inputs, outputs, stream, batch_size=1):
    # Transfer input data to the GPU.
    [cuda.memcpy_htod_async(inp.device, inp.host, stream) for inp in inputs]
    # [cuda.memcpy_htod(inp.device, inp.host) for inp in inputs]
    # Run inference.
    tic = time.time()
    context.execute_async(batch_size=batch_size, bindings=bindings, stream_handle=stream.handle)
    toc = time.time()
    dur = toc - tic
    # context.execute(batch_size=batch_size, bindings=bindings)
    # Transfer predictions back from the GPU.
    [cuda.memcpy_dtoh_async(out.host, out.device, stream) for out in outputs]
    # [cuda.memcpy_dtoh(out.host, out.device) for out in outputs]
    # Synchronize the stream
    stream.synchronize()
    # Return only the host outputs.

    return [out.host for out in outputs], dur


engine = trt_init()
# inputs, outputs, bindings, stream = allocate_buffers(engine)
inputs = []
outputs = []
bindings = []
stream = cuda.Stream()
for binding in engine:
    size = trt.volume(engine.get_binding_shape(binding)) * 1 #engine.max_batch_size
    dtype = trt.nptype(engine.get_binding_dtype(binding))
    # Allocate host and device buffers
    host_mem = cuda.pagelocked_empty(size, dtype)
    device_mem = cuda.mem_alloc(host_mem.nbytes)
    # Append the device buffer to device bindings.
    bindings.append(int(device_mem))
    # Append to the appropriate list.
    if engine.binding_is_input(binding):
        inputs.append(HostDeviceMem(host_mem, device_mem))
    else:
        outputs.append(HostDeviceMem(host_mem, device_mem))
context = engine.create_execution_context()

In [8]:
from sklearn.model_selection import train_test_split
data = np.load('mels_full.npz', allow_pickle=True)
aX, ay = data['x'], data['y']
aX_tr, aX_te, ay_tr, ay_te = train_test_split(aX, ay, test_size=0.2, shuffle=True, stratify=ay, random_state=20)
ay_te = np.where(ay_te == 'healthy', 0, 1)

In [9]:
aX_te.shape

(472, 1, 80, 401)

In [8]:
# audio_input, image_input = torch.ones(1,1,80,401), torch.ones([1, 3, 224, 224])
a_input = aX_te
 
numpy_array_input = a_input
hosts = [input.host for input in inputs]
trt_types = [trt.int32, trt.int32]
 
for numpy_array, host, trt_type in zip(numpy_array_input, hosts, trt_types):
    numpy_array = np.asarray(numpy_array).astype(trt.nptype(trt_type)).ravel()
    np.copyto(host, numpy_array)

In [14]:
# ?? -> dataload
# 생성한 numpy array를 TensorRT가 읽을 수 있는 데이터로 변환 후 input의 입력으로 할당합니다.
audio_input, image_input = torch.ones(1,1,80,401), torch.ones([1, 3, 224, 224])
a_input, i_input = audio_input.numpy(), image_input.numpy()
 
numpy_array_input = [a_input, i_input]
hosts = [input.host for input in inputs]
trt_types = [trt.int32, trt.int32]
 
for numpy_array, host, trt_type in zip(numpy_array_input, hosts, trt_types):
    numpy_array = np.asarray(numpy_array).astype(trt.nptype(trt_type)).ravel()
    np.copyto(host, numpy_array)

tic = time.time()    
trt_outputs = do_inference(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)
toc = time.time()
dur = toc - tic

In [10]:
# tensorrt batch infer
acc = []
trt_time = []
confusion_matrix = np.zeros((2, 2) ,int)

for data, label in zip(aX_te, ay_te):
    inputs[0].host = np.ascontiguousarray(data)
    trt_outputs, dur = do_inference(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream,
                        batch_size=1)
    acc.append(np.argmax(trt_outputs[1])==label)
    confusion_matrix[label, np.argmax(trt_outputs[1])] += 1
    trt_time.append(dur)
print(np.mean(trt_time))
print(np.mean(acc))

0.00020793736991235766
0.9427966101694916


In [11]:
confusion_matrix

array([[435,   4],
       [ 23,  10]])

In [13]:
inputs[0].host.shape, 80*401*64

((32080,), 2053120)

In [108]:
inputs[0].device

In [25]:
import time
tic = time.time()
trt_outputs = do_inference(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream)
toc = time.time()
dur = toc - tic
print(dur)

0.15573382377624512


In [18]:
trt_outputs[1].shape

(32,)

In [11]:
# tensorrt Latency
trt_time = []
for i in range(1000):
    trt_outputs, dur = do_inference(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream,
                        batch_size=1)
    trt_time.append(dur)
print(np.mean(trt_time))

0.0003384406566619873


In [40]:
# tensorrt Latency
trt_time = []
for i in range(1000):
    trt_outputs, dur = do_inference(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream,
                        batch_size=1)
    trt_time.append(dur)
print(np.mean(trt_time))

0.00021509265899658203


In [27]:
inputs[0].host.shape, 80*401*32

((1026560,), 1026560)

In [71]:
# tensorrt Throughput - mmnet, a_model
optimal_batch_size = 1
repetitions=100
total_time = 0
trt_time = []
with torch.no_grad():
    for rep in range(repetitions):
        _, dur = do_inference(
                        context=context,
                        bindings=bindings,
                        inputs=inputs,
                        outputs=outputs,
                        stream=stream,
#                         batch_size=64
                        )
        curr_time = dur
        total_time += curr_time
        trt_time.append(dur)
Throughput = (repetitions*optimal_batch_size)/total_time
print('Final Throughput : ',Throughput)
print(np.mean(trt_time))

Final Throughput :  4542.682306051055
0.00022013425827026366
